In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kashish8610","key":"204e556eb356ef070db999616ee91688"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d amananandrai/ag-news-classification-dataset

 70% 8.00M/11.4M [00:00<00:00, 83.2MB/s]
100% 11.4M/11.4M [00:00<00:00, 55.9MB/s]


In [5]:
!unzip -q ag-news-classification-dataset.zip

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,recall_score,accuracy_score,precision_score
from mlxtend.plotting import plot_confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout,GlobalAveragePooling1D,Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.utils import plot_model

In [9]:
TRAIN="/content/train.csv"
TEST="/content/test.csv"

In [10]:
data=pd.read_csv(TRAIN)
testdata=pd.read_csv(TEST)
data.shape,testdata.shape

((120000, 3), (7600, 3))

In [11]:
data.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [12]:
data['Class Index'].value_counts()

4    30000
3    30000
2    30000
1    30000
Name: Class Index, dtype: int64

In [14]:
#Combining Title and Description --- >
X_train=data['Title']+' '+data['Description']
y_train=data['Class Index'].apply(lambda x:x-1).values

In [15]:
testdata.head()

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [16]:
X_test=testdata['Title']+' '+testdata['Description']
y_test=testdata['Class Index'].apply(lambda x:x-1).values

In [18]:
#Max Lenghth Of Sentences in training data-
maxlen=X_train.map(lambda x:len(x.split())).max()

In [19]:
maxlen

177

In [21]:
vocab_size=12000
embed_size=32
#Creating and Fitting Tokenizers
tokenizer=Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train.values)

#Tokenize Data
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)


#Pad Sequences to the max length
X_train=pad_sequences(X_train,maxlen=maxlen)
X_test=pad_sequences(X_test,maxlen=maxlen)

In [22]:
model=Sequential()
model.add(Embedding(vocab_size,embed_size,input_length=maxlen))
model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(GlobalAveragePooling1D())
model.add(Dense(1024))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Dropout(0.25))
model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Dropout(0.25))
model.add(Dense(64))
model.add(Dropout(0.25))
model.add(Dense(4,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 177, 32)           384000    
_________________________________________________________________
bidirectional (Bidirectional (None, 177, 512)          591872    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 177, 256)          656384    
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              263168    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               5

In [23]:
callbacks=[EarlyStopping(
    monitor='val_accuracy',
    min_delta=1e-4,
    patience=4,
    verbose=1
),
ModelCheckpoint(filepath='weights.h5',
                monitor='val_accuracy',
                mode='max',
                save_best_only=True,
                save_weights_only=True,
                verbose=1)]
                

In [24]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
model.fit(X_train,y_train,batch_size=256,validation_data=(X_test,y_test),epochs=25,callbacks=callbacks)

Epoch 1/25
469/469 [==============================] - 91s 171ms/step - loss: 0.9260 - accuracy: 0.5636 - val_loss: 0.2990 - val_accuracy: 0.9087

Epoch 00001: val_accuracy improved from -inf to 0.90868, saving model to weights.h5
Epoch 2/25
469/469 [==============================] - 80s 171ms/step - loss: 0.2445 - accuracy: 0.9222 - val_loss: 0.2804 - val_accuracy: 0.9082

Epoch 00002: val_accuracy did not improve from 0.90868
Epoch 3/25
469/469 [==============================] - 80s 171ms/step - loss: 0.2007 - accuracy: 0.9347 - val_loss: 0.2941 - val_accuracy: 0.9054

Epoch 00003: val_accuracy did not improve from 0.90868
Epoch 4/25
469/469 [==============================] - 80s 171ms/step - loss: 0.1693 - accuracy: 0.9423 - val_loss: 0.2715 - val_accuracy: 0.9062

Epoch 00004: val_accuracy did not improve from 0.90868
Epoch 5/25
469/469 [==============================] - 80s 171ms/step - loss: 0.1501 - accuracy: 0.9485 - val_loss: 0.2959 - val_accuracy: 0.9047

Epoch 00005: val_accu

In [26]:
model.load_weights('weights.h5')
model.save('model.hdf5')

In [34]:
def modelDemo(news_text):

  #News Labels
  labels = ['World News', 'Sports News', 'Business News', 'Science-Technology News']

  test_seq = pad_sequences(tokenizer.texts_to_sequences(news_text), maxlen=maxlen)

  test_preds = [labels[np.argmax(i)] for i in model.predict(test_seq)]

  for news, label in zip(news_text, test_preds):
      # print('{} - {}'.format(news, label))
      print('{} - {}'.format(colored(news, 'green'), colored(label, 'red')))

In [35]:
from termcolor import colored

In [36]:
modelDemo(['New evidence of virus risks from wildlife trade'])

New evidence of virus risks from wildlife trade - Science-Technology News


In [42]:
testdata['Title'][6]

'Loosing the War on Terrorism'

In [39]:
modelDemo(['The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com)'])

The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) - Science-Technology News


In [43]:
modelDemo(['Loosing the War on Terrorism'])

Loosing the War on Terrorism - Science-Technology News


In [44]:
modelDemo(['Coronavirus: Bank pumps £100bn into UK economy to aid recovery'])

Coronavirus: Bank pumps £100bn into UK economy to aid recovery - Business News


In [45]:
modelDemo(['David Luiz’s future with Arsenal to be decided this week'])

David Luiz’s future with Arsenal to be decided this week - Science-Technology News


In [47]:
preds = [np.argmax(i) for i in model.predict(X_test)]

In [48]:
print("Recall of the model is {:.2f}".format(recall_score(y_test, preds, average='micro')))
print("Precision of the model is {:.2f}".format(precision_score(y_test, preds, average='micro')))
print("Accuracy of the model is {:.2f}".format(accuracy_score(y_test, preds)))

Recall of the model is 0.91
Precision of the model is 0.91
Accuracy of the model is 0.91
